In [4]:
%pip install requests pandas google-cloud-bigquery apache-airflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Ale\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [8]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

In [19]:
# Constants
CLOUDINARY_API_URL = "https://api.cloudinary.com/v1_1/dqyf4lrlo/resources/search"
MONGODB_API_URL = "https://bahela-admin.vercel.app/api/sessions"
CLOUDINARY_AUTH = ("214285446753493", "_Lwj_-7ml1KqQAe4Oadpaxoqhf4")
today = datetime.strptime("2024-11-18", "%Y-%m-%d")
START_DATE = (today - timedelta(days=14)).strftime("%Y-%m-%d")
END_DATE = (today - timedelta(days=1)).strftime("%Y-%m-%d")

In [20]:
def fetch_cloudinary_data(start_date, end_date):
    all_data = []
    next_cursor = None
    while True:
        # Modify the expression to include the date range
        params = {
            "expression": f"created_at:[{start_date} TO {end_date}]",
            "max_results": 50,
        }
        if next_cursor:
            params["next_cursor"] = next_cursor
        response = requests.get(CLOUDINARY_API_URL, auth=CLOUDINARY_AUTH, params=params)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data.get("resources", []))
            next_cursor = data.get("next_cursor")
            if not next_cursor:  # Break if no more pages
                break
        else:
            print(f"Failed to fetch Cloudinary data: {response.text}")
            break
    return pd.DataFrame(all_data)

# Fetch Cloudinary data for the last three months
cloudinary_data = fetch_cloudinary_data(START_DATE, END_DATE)
cloudinary_data.head()

,asset_id,public_id,asset_folder,filename,display_name,format,version,resource_type,type,created_at,...,aspect_ratio,pixels,url,secure_url,status,access_mode,access_control,etag,created_by,uploaded_by
0,72caf305164be1f2b9f642995c2df9a1,pg7kl1-7,,pg7kl1-7,pg7kl1-7,png,1731859713,image,upload,2024-11-17T16:08:33+00:00,...,0.66543,779040,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,89b7b7d42065e76dd7d0c0e64032a9d2,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
1,235db9257c3b1dce5c7a4a1afd8440d5,pg7kl1-6,,pg7kl1-6,pg7kl1-6,png,1731859711,image,upload,2024-11-17T16:08:31+00:00,...,0.33179,388438,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,cdb6bcae7d3254a48240de20c6c7ea97,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
2,f6cec4bd43efe0ce8ce5b6ffc621bf2c,pg7kl1-5,,pg7kl1-5,image-6,png,1731859611,image,upload,2024-11-17T16:06:51+00:00,...,1.00000,409600,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,95f6e89a9493dd20f44d235837044b1f,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
3,bfdf28c08db65791b82efacdb73cf697,pg7kl1-4,,pg7kl1-4,image-5,png,1731859610,image,upload,2024-11-17T16:06:50+00:00,...,1.00000,409600,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,ba6300afe1024c6a424be41d5b37761a,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
4,153ef98a63d0c7932012c5ff6320a5fc,pg7kl1-3,,pg7kl1-3,image-4,png,1731859609,image,upload,2024-11-17T16:06:49+00:00,...,1.00000,409600,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,dc500c5611f68e3684731c7e1d3de46e,{'access_key': '214285446753493'},{'access_key': '214285446753493'}


In [21]:
def fetch_mongodb_data(start_date, end_date):
    # Parameters with incremented end_date
    params = {"startDate": start_date, "endDate": end_date}
    response = requests.get(MONGODB_API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data.get("appStates", []))
    else:
        print(f"Failed to fetch MongoDB data: {response.text}")
        return pd.DataFrame()

# Fetch MongoDB data for yesterday only
mongodb_data = fetch_mongodb_data(START_DATE, END_DATE)
mongodb_data.head() 

,progress,options,payment,result,_id,id,page,__v
0,"{'appStartTime': '2024-11-17T01:03:07.325Z', '...","{'lang': 'id', 'filterFn': 'transformToJelaga'...","{'amount': 60000, 'orderId': 'order-pg7kl1-173...",{'capturedImageUrls': ['blob:file:///ab25beea-...,673a15020d942cbf8f11d004,pg7kl1,Result,0
1,"{'appStartTime': '2024-11-17T01:03:07.325Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 60000, 'orderId': 'order-a8k7ni-173...",{'capturedImageUrls': ['blob:file:///b7cf9c2e-...,673a13b00d942cbf8f11d001,a8k7ni,Result,0
2,"{'appStartTime': '2024-11-17T01:03:07.325Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 35000, 'orderId': 'order-44u1tm-173...",{'capturedImageUrls': ['blob:file:///d0ed7aed-...,673a0f5cadb4ce2070345c43,44u1tm,Result,0
3,"{'appStartTime': '2024-11-17T01:03:07.325Z', '...","{'lang': 'id', 'filterFn': 'transformToJelaga'...","{'amount': 35000, 'orderId': 'order-2xucap-173...",{'capturedImageUrls': ['blob:file:///b0c3c1b3-...,673a0d134502281bd7496b61,2xucap,Result,0
4,"{'appStartTime': '2024-11-17T01:03:07.325Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 35000, 'orderId': 'order-257ii6-173...",{'capturedImageUrls': ['blob:file:///7e98d26f-...,673a0b324502281bd7496b5e,257ii6,Result,0


In [24]:
# Section 1: Create joined_data
def create_joined_data(cloudinary_df, mongodb_df):

    # Filter MongoDB data
    filtered_mongodb = mongodb_df[
        (mongodb_df["page"] == "Result") &
        (mongodb_df["payment"].apply(lambda x: x.get("paid", False)))
    ]
    
    # Flatten progress, including start and end time
    def flatten_progress(progress):
        return {k: v for k, v in progress.items() if "error" not in k}
    
    filtered_mongodb["progress"] = filtered_mongodb["progress"].apply(flatten_progress)
    filtered_mongodb = filtered_mongodb.assign(
        app_start_time=filtered_mongodb["progress"].apply(lambda x: x.get("appStartTime")),
        start_time=filtered_mongodb["progress"].apply(lambda x: x.get("startTime")),
        end_time=filtered_mongodb["progress"].apply(lambda x: x.get("endTime"))
    )
    
    # Ensure start_time and end_time are datetime objects
    filtered_mongodb["start_time"] = pd.to_datetime(filtered_mongodb["start_time"], errors='coerce')
    filtered_mongodb["end_time"] = pd.to_datetime(filtered_mongodb["end_time"], errors='coerce')

    # Calculate session duration (end_time - start_time)
    filtered_mongodb["duration"] = (filtered_mongodb["end_time"] - filtered_mongodb["start_time"]).dt.total_seconds()

    # Flatten options and uniform column names
    filtered_mongodb = filtered_mongodb.assign(
        lang=filtered_mongodb["options"].apply(lambda x: x.get("lang")),
        filter_fn=filtered_mongodb["options"].apply(lambda x: x.get("filterFn")),
        copies=filtered_mongodb["options"].apply(lambda x: x.get("copies")),
        use_retry=filtered_mongodb["options"].apply(lambda x: x.get("useRetryMode")),  # Added retryCount
        retry_count=filtered_mongodb["options"].apply(lambda x: x.get("retryCount", 0) if x.get("useRetryMode", False) else 0)
    )
    
    # Flatten payment details and add discount and expiry_time
    filtered_mongodb = filtered_mongodb.assign(
        amount=filtered_mongodb["payment"].apply(lambda x: x.get("amount")),
        order_id=filtered_mongodb["payment"].apply(lambda x: x.get("orderId")),
        discount=filtered_mongodb["payment"].apply(lambda x: x.get("discount", 0)),  # Default to 0 if not found
        expiry_time=filtered_mongodb["payment"].apply(lambda x: x.get("expiryTime"))  # Assuming it's a timestamp or datetime string
    )
    
    # Extract and link uploadedImageUrls with Cloudinary data
    def map_uploaded_images(row):
        # Filter Cloudinary data for matching public_ids and ensure public_id includes MongoDB id
        matched_images = cloudinary_df[cloudinary_df['public_id'].str.contains(row['id'])]

        # Return required fields as a list of dictionaries
        result = matched_images[[
            "asset_id", "public_id", "resource_type", "secure_url", "bytes", 
            "width", "height", "aspect_ratio", "pixels", "created_at"
        ]].to_dict("records")

        return result

    filtered_mongodb["uploaded_images"] = filtered_mongodb.apply(map_uploaded_images, axis=1)
    

    # Calculate bytes and pixels summary for each session
    def calculate_summary(row):
        # Extract public_ids from uploaded_images
        uploaded_ids = [img["public_id"] for img in row["uploaded_images"]]
        
        # Filter Cloudinary data for matching public_ids
        related_resources = cloudinary_df[cloudinary_df["public_id"].isin(uploaded_ids)]
        
        # Calculate totals and averages
        total_bytes = related_resources["bytes"].sum()
        avg_bytes = related_resources["bytes"].mean() if not related_resources.empty else 0
        total_pixels = related_resources["pixels"].sum()
        avg_pixels = related_resources["pixels"].mean() if not related_resources.empty else 0
        
        # Return a pandas Series with calculated values
        return pd.Series({
            "total_bytes": total_bytes,
            "avg_bytes": avg_bytes,
            "total_pixels": total_pixels,
            "avg_pixels": avg_pixels
        })

    bytes_pixels_summary = filtered_mongodb.apply(calculate_summary, axis=1)
    filtered_mongodb = pd.concat([filtered_mongodb, bytes_pixels_summary], axis=1)
    
    # Create final joined_data
    joined_data = filtered_mongodb[[
        "_id", "id", "app_start_time", "start_time", "end_time", "duration", "lang", "copies",
        "filter_fn", "amount", "order_id", "discount", "expiry_time", "use_retry", "retry_count", 
        "uploaded_images", "total_bytes", "avg_bytes", "total_pixels", "avg_pixels"
    ]].rename(columns={
        "_id": "id", 
        "id": "session_id", 
        "amount": "transaction_amount",
        "retry_count": "retry_attempts"
    })
    
    return joined_data

# Call the function to create joined_data
joined_data = create_joined_data(cloudinary_data, mongodb_data)
joined_data.head()  # Preview the result

C:\Users\Ale\AppData\Local\Temp\ipykernel_1228\988797939.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_mongodb["progress"] = filtered_mongodb["progress"].apply(flatten_progress)


,id,session_id,app_start_time,start_time,end_time,duration,lang,copies,filter_fn,transaction_amount,order_id,discount,expiry_time,use_retry,retry_attempts,uploaded_images,total_bytes,avg_bytes,total_pixels,avg_pixels
0,673a15020d942cbf8f11d004,pg7kl1,2024-11-17T01:03:07.325Z,2024-11-17 16:04:17.197000+00:00,2024-11-17 16:08:33.310000+00:00,256.113,id,2,transformToJelaga,60000,order-pg7kl1-1731859480051,0,2024-11-17T23:06:41.000Z,True,1,[{'asset_id': '72caf305164be1f2b9f642995c2df9a...,3431733.0,428966.625,3625078.0,453134.75
1,673a13b00d942cbf8f11d001,a8k7ni,2024-11-17T01:03:07.325Z,2024-11-17 15:56:53.680000+00:00,2024-11-17 16:02:54.418000+00:00,360.738,id,2,transformToMentari,60000,order-a8k7ni-1731859100658,0,2024-11-17T23:00:21.000Z,True,1,[{'asset_id': 'a564630a1ca4c0b90b4ad78f57bc9ef...,7215743.0,901967.875,3625078.0,453134.75
2,673a0f5cadb4ce2070345c43,44u1tm,2024-11-17T01:03:07.325Z,2024-11-17 15:40:47.488000+00:00,2024-11-17 15:44:26.853000+00:00,219.365,id,1,transformToMentari,35000,order-44u1tm-1731858070486,0,2024-11-17T22:43:11.000Z,False,0,[{'asset_id': '3cf883cea4e3add5fe6f3cd4bc284dc...,6509511.0,813688.875,3625078.0,453134.75
3,673a0d134502281bd7496b61,2xucap,2024-11-17T01:03:07.325Z,2024-11-17 15:31:45.383000+00:00,2024-11-17 15:34:42.618000+00:00,177.235,id,1,transformToJelaga,35000,order-2xucap-1731857520042,0,2024-11-17T22:34:01.000Z,True,2,[{'asset_id': 'ed16c5b8811075f6d4540e3e1e6ff6e...,3200536.0,400067.000,3625078.0,453134.75
4,673a0b324502281bd7496b5e,257ii6,2024-11-17T01:03:07.325Z,2024-11-17 15:23:22.821000+00:00,2024-11-17 15:26:40.506000+00:00,197.685,id,1,transformToMentari,35000,order-257ii6-1731857040815,0,2024-11-17T22:26:02.000Z,False,0,[{'asset_id': '90e88b90b9ec6d6271c5bf5393e2b0a...,6933431.0,866678.875,3625078.0,453134.75


In [23]:
# Section 2: Create fact_sessions
def create_fact_sessions(joined_data):
    fact_sessions = joined_data[[
        "session_id", "start_time", "duration", "transaction_amount", "discount", "retry_attempts", "copies",
        "total_bytes", "avg_bytes", "total_pixels", "avg_pixels"
    ]]
    
    return fact_sessions

# Create fact_sessions from joined_data
fact_sessions = create_fact_sessions(joined_data)
fact_sessions.head()  # Preview the result

,session_id,start_time,duration,transaction_amount,discount,retry_attempts,copies,total_bytes,avg_bytes,total_pixels,avg_pixels
0,pg7kl1,2024-11-17 16:04:17.197000+00:00,256.113,60000,0,1,2,3431733.0,428966.625,3625078.0,453134.75
1,a8k7ni,2024-11-17 15:56:53.680000+00:00,360.738,60000,0,1,2,7215743.0,901967.875,3625078.0,453134.75
2,44u1tm,2024-11-17 15:40:47.488000+00:00,219.365,35000,0,0,1,6509511.0,813688.875,3625078.0,453134.75
3,2xucap,2024-11-17 15:31:45.383000+00:00,177.235,35000,0,2,1,3200536.0,400067.000,3625078.0,453134.75
4,257ii6,2024-11-17 15:23:22.821000+00:00,197.685,35000,0,0,1,6933431.0,866678.875,3625078.0,453134.75


In [17]:
# Section 3: Create dim_sessions
def create_dim_sessions(joined_data):
    dim_sessions = joined_data[[
        "session_id", "app_start_time", "start_time", "end_time",
        "use_retry", "lang", "filter_fn", "order_id", "expiry_time", "uploaded_images"
    ]]
    
    # Convert 'expiry_time' and 'app_start_time' to datetime if needed
    dim_sessions["expiry_time"] = pd.to_datetime(dim_sessions["expiry_time"], errors='coerce')
    dim_sessions["app_start_time"] = pd.to_datetime(dim_sessions["app_start_time"], errors='coerce')

    return dim_sessions

# Create dim_sessions from joined_data
dim_sessions = create_dim_sessions(joined_data)
dim_sessions.head()  # Preview the result


C:\Users\Ale\AppData\Local\Temp\ipykernel_1228\2771637358.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_sessions["expiry_time"] = pd.to_datetime(dim_sessions["expiry_time"], errors='coerce')
C:\Users\Ale\AppData\Local\Temp\ipykernel_1228\2771637358.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_sessions["app_start_time"] = pd.to_datetime(dim_sessions["app_start_time"], errors='coerce')


,session_id,app_start_time,start_time,end_time,use_retry,lang,filter_fn,order_id,expiry_time,uploaded_images
0,pg7kl1,2024-11-17 01:03:07.325000+00:00,2024-11-17 16:04:17.197000+00:00,2024-11-17 16:08:33.310000+00:00,True,id,transformToJelaga,order-pg7kl1-1731859480051,2024-11-17 23:06:41+00:00,[{'asset_id': '72caf305164be1f2b9f642995c2df9a...
1,a8k7ni,2024-11-17 01:03:07.325000+00:00,2024-11-17 15:56:53.680000+00:00,2024-11-17 16:02:54.418000+00:00,True,id,transformToMentari,order-a8k7ni-1731859100658,2024-11-17 23:00:21+00:00,[{'asset_id': 'a564630a1ca4c0b90b4ad78f57bc9ef...
2,44u1tm,2024-11-17 01:03:07.325000+00:00,2024-11-17 15:40:47.488000+00:00,2024-11-17 15:44:26.853000+00:00,False,id,transformToMentari,order-44u1tm-1731858070486,2024-11-17 22:43:11+00:00,[{'asset_id': '3cf883cea4e3add5fe6f3cd4bc284dc...
3,2xucap,2024-11-17 01:03:07.325000+00:00,2024-11-17 15:31:45.383000+00:00,2024-11-17 15:34:42.618000+00:00,True,id,transformToJelaga,order-2xucap-1731857520042,2024-11-17 22:34:01+00:00,[{'asset_id': 'ed16c5b8811075f6d4540e3e1e6ff6e...
4,257ii6,2024-11-17 01:03:07.325000+00:00,2024-11-17 15:23:22.821000+00:00,2024-11-17 15:26:40.506000+00:00,False,id,transformToMentari,order-257ii6-1731857040815,2024-11-17 22:26:02+00:00,[{'asset_id': '90e88b90b9ec6d6271c5bf5393e2b0a...


In [18]:
def create_combined_summary(joined_data):
    # Extract date from start_time for grouping
    joined_data['date'] = joined_data['start_time'].dt.date
    
    # Create marketing & sales summary
    marketing_sales_summary = joined_data.groupby('date').agg(
        transaction_amount=('transaction_amount', 'sum'),
        total_discount=('discount', 'sum'),
        total_transactions=('transaction_amount', 'count')
    ).reset_index()

    # Create user behavior facts summary
    user_behavior_facts = joined_data.groupby('date').agg(
        total_duration=('duration', 'sum'),
        avg_duration=('duration', 'mean'),
        total_copies=('copies', 'sum'),
        avg_copies=('copies', 'mean')
    ).reset_index()

    # Create operations summary
    operations_summary = joined_data.groupby('date').agg(
        total_bytes=('total_bytes', 'sum'),
        avg_bytes=('avg_bytes', 'mean'),
        total_pixels=('total_pixels', 'sum'),
        avg_pixels=('avg_pixels', 'mean')
    ).reset_index()

    # Combine all summaries into one table by merging on the 'date' column
    combined_summary = marketing_sales_summary.merge(user_behavior_facts, on='date', how='outer')
    combined_summary = combined_summary.merge(operations_summary, on='date', how='outer')

    return combined_summary

# Call the function to create the combined summary
combined_summary = create_combined_summary(joined_data)
combined_summary.head()

,date,transaction_amount,total_discount,total_transactions,total_duration,avg_duration,total_copies,avg_copies,total_bytes,avg_bytes,total_pixels,avg_pixels
0,2024-11-17,2265000,0,59,15258.694,258.621932,67,1.135593,371293524.0,786638.822034,213879602.0,453134.75
